Задание:

Сгенерировать 10 изображений с помощью StyleGan2
Построить функцию интерполяции между двумя изображениями
Сделать замену лиц с помощью StyleGan2:
Открыть и нормализовать изображения
Инициировать функию потерь
Получить стартовую (любым из способов: e4e, средний вектор, инетрполяция)
Применить разные функции потерь
Результат оформить в виде сконкатенированной картинки из исходного изображения, источника и результата замены лица

In [37]:
!pip install ninja

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.7/180.7 kB 6.3 MB/s eta 0:00:00


In [2]:
# Клонируем репозиторий StyleGAN2-ADA-PyTorch
!git clone https://github.com/NVlabs/stylegan2-ada-pytorch.git
%cd stylegan2-ada-pytorch

# Устанавливаем зависимости
!pip install torch torchvision lpips ninja

Cloning into 'stylegan2-ada-pytorch'...
remote: Enumerating objects: 131, done.
remote: Counting objects: 100% (2/2), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 131 (delta 0), reused 0 (delta 0), pack-reused 129 (from 2)
Receiving objects: 100% (131/131), 1.13 MiB | 15.64 MiB/s, done.
Resolving deltas: 100% (57/57), done.
/content/stylegan2-ada-pytorch/stylegan2-ada-pytorch
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.7/180.7 kB 7.7 MB/s eta 0:00:00


In [1]:
# FFHQ 1024x1024
!mkdir -p models
!curl -L https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada-pytorch/pretrained/ffhq.pkl -o models/stylegan2-ffhq.pkl

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  363M  100  363M    0     0  21.5M      0  0:00:16  0:00:16 --:--:-- 25.0M


In [3]:
!git clone https://github.com/NVlabs/stylegan2-ada-pytorch.git

Cloning into 'stylegan2-ada-pytorch'...
remote: Enumerating objects: 131, done.
remote: Counting objects: 100% (2/2), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 131 (delta 0), reused 0 (delta 0), pack-reused 129 (from 2)
Receiving objects: 100% (131/131), 1.13 MiB | 4.02 MiB/s, done.
Resolving deltas: 100% (57/57), done.
/content/stylegan2-ada-pytorch


In [4]:
import sys
sys.path.append('/content/stylegan2-ada-pytorch')

In [18]:
!pip install lpips

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 kB 2.3 MB/s eta 0:00:00


In [54]:
import torch, os
import pickle
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from torchvision import transforms
import lpips
import cv2

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
results_dir = '/content/results'
os.makedirs(results_dir, exist_ok=True)

# Загрузка StyleGAN2-ADA
with open('/content/models/stylegan2-ffhq.pkl', 'rb') as f:
    G = pickle.load(f)['G_ema'].to(device)

# Трансформ изображений
transform = transforms.Compose([
    transforms.Resize((512,512)),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3,[0.5]*3)
])

def tensor_to_np(img_tensor):
    img_np = img_tensor[0].permute(1,2,0).cpu().numpy()
    img_np = ((img_np+1)*127.5).astype(np.uint8)
    return img_np

# Загружаем target и source
target_path = '/content/results/gen_0.png'
source_path = '/content/results/gen_7.png'
target_img = transform(Image.open(target_path).convert('RGB')).unsqueeze(0).to(device)
source_img = transform(Image.open(source_path).convert('RGB')).unsqueeze(0).to(device)

# LPIPS и MSE loss
lpips_loss = lpips.LPIPS(net='vgg').to(device)
mse_loss = torch.nn.MSELoss()

# Инициализация latent w через среднее генератора
with torch.no_grad():
    sample_z = torch.randn(4096, G.z_dim).to(device)
    w_samples = G.mapping(sample_z, None)
    w_avg = w_samples.mean(0, keepdim=True)[:,0,:]

# Функция оптимизации latent
def invert_image(img_tensor, steps=100, lr=0.05):
    w = w_avg.clone().detach().requires_grad_(True)
    optimizer = torch.optim.Adam([w], lr=lr)
    for step in range(steps):
        optimizer.zero_grad()
        gen = G(w, None, truncation_psi=0.7)
        gen_resized = torch.nn.functional.interpolate(gen, size=(512,512), mode='bilinear')
        loss = mse_loss(gen_resized, img_tensor) + lpips_loss(gen_resized, img_tensor).mean()
        loss.backward()
        optimizer.step()
    return w.detach()

w_target = invert_image(target_img)
w_source = invert_image(source_img)

# Генерируем изображения из w_target и w_source
with torch.no_grad():
    img_target = G(w_target, None, truncation_psi=0.7)
    img_source = G(w_source, None, truncation_psi=0.7)
    img_target_np = tensor_to_np(torch.nn.functional.interpolate(img_target, size=(512,512), mode='bilinear'))
    img_source_np = tensor_to_np(torch.nn.functional.interpolate(img_source, size=(512,512), mode='bilinear'))

# Создаем маску лица (используем OpenCV haarcascade для простоты)
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
gray = cv2.cvtColor(img_target_np, cv2.COLOR_RGB2GRAY)
faces = face_cascade.detectMultiScale(gray, 1.1, 4)
mask = np.zeros_like(gray)
for (x,y,w,h) in faces:
    mask[y:y+h, x:x+w] = 1
mask = np.stack([mask]*3, axis=2)  # RGB

Setting up [LPIPS] perceptual loss: trunk [vgg], v[0.1], spatial [off]


/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Loading model from: /usr/local/lib/python3.12/dist-packages/lpips/weights/v0.1/vgg.pth
Setting up PyTorch plugin "upfirdn2d_plugin"... Failed!
Setting up PyTorch plugin "upfirdn2d_plugin"... Failed!
Setting up PyTorch plugin "upfirdn2d_plugin"... Failed!
Setting up PyTorch plugin "upfirdn2d_plugin"... Failed!
Setting up PyTorch plugin "upfirdn2d_plugin"... Failed!
Setting up PyTorch plugin "upfirdn2d_plugin"... Failed!
Setting up PyTorch plugin "upfirdn2d_plugin"... Failed!
Setting up PyTorch plugin "upfirdn2d_plugin"... Failed!
Setting up PyTorch plugin "upfirdn2d_plugin"... Failed!
Setting up PyTorch plugin "upfirdn2d_plugin"... Failed!
Setting up PyTorch plugin "upfirdn2d_plugin"... Failed!
Setting up PyTorch plugin "upfirdn2d_plugin"... Failed!
Setting up PyTorch plugin "upfirdn2d_plugin"... Failed!
Setting up PyTorch plugin "upfirdn2d_plugin"... Failed!
Setting up PyTorch plugin "upfirdn2d_plugin"... Failed!
Setting up PyTorch plugin "upfirdn2d_plugin"... Failed!
Setting up PyTorc

In [60]:
# Ввод альфа в консоли
try:
    alpha = float(input("Введите значение alpha (0.0–1.0, где 0=только target, 1=только source): "))
    if not (0.0 <= alpha <= 1.0):
        print("Alpha вне диапазона 0–1, будет использовано значение 0.5")
        alpha = 0.5
except:
    print("Некорректный ввод, будет использовано значение 0.5")
    alpha = 0.5

# Генерация face-swap по маске лица
result_np = img_target_np.copy()
# линейная смесь только по маске лица
result_np[mask==1] = ((1-alpha)*img_target_np[mask==1] + alpha*img_source_np[mask==1]).astype(np.uint8)

# Отображение и сохранение результата
combined = np.concatenate([img_target_np, img_source_np, result_np], axis=1)
plt.figure(figsize=(15,5))
plt.imshow(combined)
plt.axis('off')
plt.show()

plt.imsave(f'{results_dir}/final_face_swap_alpha_{int(alpha*100)}.png', combined)
print(f"Face-swap с alpha={alpha} сохранен:", f'{results_dir}/final_face_swap_alpha_{int(alpha*100)}.png')


Введите значение alpha (0.0–1.0, где 0=только target, 1=только source): 0.15
Face-swap с alpha=0.15 сохранен: /content/results/final_face_swap_alpha_15.png
